In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
# Xử lý dữ liệu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Kalman Filter (filterpy rất phổ biến)
!pip install filterpy
from filterpy.kalman import KalmanFilter

# HMM (hmmlearn khá chuẩn cho HMM cơ bản)
!pip install hmmlearn
from hmmlearn import hmm

# SLDS (sử dụng pykalman hoặc pykalman + custom HMM)
!pip install pykalman
from pykalman import KalmanFilter as PyKalmanFilter

# Xử lý tín hiệu (cho ECG, cảm biến,…)
from scipy import signal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=96d992850ae4334992b53225e2dd409cfd156b300517a423ee96f8d39c63c49e
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/HMM/test.csv'
df = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,STANDING
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,2,STANDING
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,2,STANDING


In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
import numpy as np

# Lấy dữ liệu cảm biến và nhãn
X = df.drop(columns=['Activity', 'subject'])
y = df['Activity']

# Mã hóa nhãn hoạt động
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# === Train Gaussian Mixture Model (GMM) ===
gmm = GaussianMixture(n_components=6, covariance_type='full', random_state=42)
gmm.fit(X)

# === Dự đoán chế độ (mode) cho mỗi đoạn dữ liệu (ẩn) ===
gmm_hidden_states = gmm.predict(X)

# === Train HMM dựa trên trạng thái từ GMM ===
model = hmm.GaussianHMM(n_components=6, covariance_type='diag', n_iter=1000, random_state=42)
model.fit(X)

# Dự đoán chuỗi trạng thái ẩn (cho SLDS)
hidden_states = model.predict(X)

print("Ví dụ chuỗi trạng thái ẩn đầu tiên:", hidden_states[:30])


Ví dụ chuỗi trạng thái ẩn đầu tiên: [5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 3 3 3 5 5 0 0 0 0 0 0]


In [ ]:
import torch
import torch.nn as nn

class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super().__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(model_dim, output_dim)

    def forward(self, x):
        # x shape: [seq_len, batch_size, input_dim]
        x = self.embedding(x)
        x = self.transformer(x)
        return self.fc_out(x)

# Khởi tạo model demo
model = SimpleTransformer(input_dim=561, model_dim=128, num_heads=4, num_layers=2, output_dim=6)


In [ ]:
import torch
import numpy as np

# Dữ liệu mẫu: 100 dòng đầu tiên từ X
X_np = df.drop(columns=["Activity", "subject"]).values[:100]

# Vì không có dữ liệu theo chuỗi thời gian thực sự, ta giả sử mỗi dòng là 1 bước trong chuỗi dài
# Giả định batch size = 1, sequence length = 100, input_dim = 561
X_tensor = torch.tensor(X_np, dtype=torch.float32).unsqueeze(0)  # shape: [1, 100, 561]


In [ ]:
# Đầu ra từ mô hình: [batch_size, seq_len, output_dim]
output = model(X_tensor)
print("Shape of output:", output.shape)  # → (1, 100, 6)
print("Output tại timestep 0:", output[0, 0])


Shape of output: torch.Size([1, 100, 6])
Output tại timestep 0: tensor([-0.7121,  1.4765,  0.5869,  0.5445, -0.1779,  0.1206],
       grad_fn=<SelectBackward0>)


In [ ]:
import torch.nn.functional as F

# Áp dụng softmax lên từng bước thời gian
probs = F.softmax(output, dim=-1)  # xác suất 6 lớp
print("Xác suất lớp tại timestep 0:", probs[0, 0])


Xác suất lớp tại timestep 0: tensor([0.0474, 0.4227, 0.1737, 0.1664, 0.0808, 0.1089],
       grad_fn=<SelectBackward0>)


In [ ]:
predicted_classes = torch.argmax(probs, dim=-1)  # shape: [1, 100]
print("Các lớp dự đoán:", predicted_classes)


Các lớp dự đoán: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
         1, 1, 3, 3]])
